In [2]:
from datasets import load_dataset

ds = load_dataset("yandex/yambda", data_dir="flat/50m", data_files="listens.parquet")
df = ds["train"].to_pandas()

In [3]:
df = df[df["played_ratio_pct"] >= 50]

In [10]:
item_vcs = df["item_id"].value_counts()
user_vcs = df["uid"].value_counts()
tmp = df.copy()
tmp = tmp[tmp["item_id"].isin(item_vcs[item_vcs > 100].index)]
tmp = tmp[tmp["uid"].isin(user_vcs[user_vcs > 30].index)]

In [14]:
tmp["item_id"].nunique()

46480

In [6]:
tmp.shape

(26667595, 6)

In [7]:
import pandas as pd
import numpy as np


def create_session_ids(df: pd.DataFrame, split_thresh=1800):
    df = df.copy()
    df["time_gap"] = df.groupby("uid")["timestamp"].diff()
    df["is_new_session"] = df["time_gap"] > split_thresh // 5
    df["user_session_id"] = df.groupby("uid")["is_new_session"].cumsum()
    df["session_id"] = df.groupby(["uid", "user_session_id"]).ngroup()
    return df.drop(columns=["time_gap", "is_new_session", "user_session_id"])


tmp = create_session_ids(tmp)

In [8]:
session_sizes = tmp.groupby("session_id").size()
tmp[tmp["session_id"].isin(session_sizes[session_sizes > 5].index)][
    "session_id"
].nunique()

1330422

In [26]:
df[df["item_id"].isin(item_vcs[item_vcs > 50].index)]["item_id"].nunique()

30055

In [4]:
tmp["session_id"].nunique()

357536